### Imports and Setup

In [1]:
import torch
import classiq

import torch.nn as nn
import torch.optim as optim
from torchinfo import summary

from models.leqm3 import linear_entanglement_r3_quantum_model
from models.qnn import execute_fn, post_process_fn, QNN

from scripts.helper import create_writer, write_train_results
from scripts.data_setup import create_mnist_dataloaders
from scripts.data_transforms import input_transform, target_transform
from scripts.train import train
from scripts.test import test

/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Authenticate Classiq
classiq.authenticate()

The current version of 'classiq' has been deprecated, and will not be supported as of 2024-01-18. Please run "pip install -U classiq" to upgrade the classiq SDK to the latest version.
/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/classiq/_internals/authentication/token_manager.py:82: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [3]:
## For setting up device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'
device

'cpu'

In [4]:
## Clear Output Files
post_process_output_file = open("post_process_output.txt", "w")
print("-----------------------------------------------------------------------------------------------------------------", file=post_process_output_file)
print("--------------------------------------------POST PROCESS OUTPUT--------------------------------------------------", file=post_process_output_file)
print("-----------------------------------------------------------------------------------------------------------------", file=post_process_output_file)
post_process_output_file.close()

test_loop_output_file = open("test_loop_output.txt", "w")
print("-----------------------------------------------------------------------------------------------------------------", file=test_loop_output_file)
print("-----------------------------------------------TEST LOOP OUTPUT--------------------------------------------------", file=test_loop_output_file)
print("-----------------------------------------------------------------------------------------------------------------", file=test_loop_output_file)
test_loop_output_file.close()

In [5]:
# HYPER PARAMETERS
_LEARNING_RATE = 1.0
BATCH_SIZE = 64
EPOCHS = 2

### Quantum Model

In [6]:
#  Create a Linear Entanglement Quantum Model for MNIST Data Classification with three linear entanglement layers of RXX, RYY, and RZZ.
quantum_model = linear_entanglement_r3_quantum_model()

In [7]:
quantum_program = classiq.synthesize(quantum_model)

In [8]:
# View Quantum Program on Classiq Platform
classiq.show(quantum_program)

Opening: https://platform.classiq.io/circuit/ded3d6f8-8b3e-445e-b3e6-39bbd5ea6371?version=0.33.0


### Quantum Neural Network

In [9]:
qnn = QNN(
    quantum_program=quantum_program,
    execute=execute_fn,
    post_process=post_process_fn,
)

gio: https://platform.classiq.io/circuit/ded3d6f8-8b3e-445e-b3e6-39bbd5ea6371?version=0.33.0: Operation not supported


In [10]:
# summary(model=qnn, input_size=(32, 16), verbose=0, col_names=["input_size", "output_size", "num_params", "trainable"], col_width=20, row_settings=["var_names"])

In [11]:
# choosing our loss function
loss_fn = nn.L1Loss()
# choosing our optimizer
optimizer = optim.SGD(qnn.parameters(), lr=_LEARNING_RATE)

### Preparing Data

In [12]:
train_dataloader, test_dataloader, class_names = create_mnist_dataloaders(
    root="data",
    transform=input_transform,
    target_transform=target_transform,
    batch_size=BATCH_SIZE,
    create_subset=True,
    subset_size=64
)

In [13]:
# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")
print(f"Our Dataset have following classes: {class_names}")

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7efbc9e5e0d0>, <torch.utils.data.dataloader.DataLoader object at 0x7efbc6efc110>)
Length of train dataloader: 1 batches of 64
Length of test dataloader: 1 batches of 64
Our Dataset have following classes: ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']


In [14]:
data, label = next(iter(train_dataloader))

print(f"Image shape: {data.shape} -> [batch_size, pixel_angle]")
print(f"Label shape: {label.shape} -> [batch_size, label_value]")

/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Image shape: torch.Size([64, 16]) -> [batch_size, pixel_angle]
Label shape: torch.Size([64, 10]) -> [batch_size, label_value]


#### Run Experiment

##### 01. Train

In [15]:
# Create a writer for tracking our experiment
writer = create_writer(experiment_name="data_0.1_percent", model_name="linear_entanglement_r3", extra=f"{EPOCHS}_epochs")

[INFO] Created SummaryWriter, saving to: runs/2024-01-04/data_0.1_percent/linear_entanglement_r3/2_epochs...


In [16]:
train_results = train(
    model = qnn, 
    data_loader = train_dataloader, 
    loss_fn = loss_fn, 
    optimizer = optimizer, 
    writer = writer, 
    epochs = EPOCHS,
    device = device
)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 0
----------


/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/models/qnn.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_g

Epoch: 1 | Train loss: 0.18140
Epoch: 1
----------


/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 2/2 [01:58<00:00, 59.27s/it]

Epoch: 2 | Train loss: 0.36204


In [17]:
# Check out the model results
print(train_results)

{'train_loss': [0.18140220642089844, 0.36204254627227783]}


In [18]:
write_train_results(experiment_name="data_0.1_percent", model_name="linear_entanglement_r3", epochs=EPOCHS, results=train_results)

In [19]:
# %load_ext tensorboard
# %tensorboard --logdir runs

##### 02. Test

In [20]:
test_results = test(
    model = qnn, 
    data_loader = test_dataloader,
    device = device
)

/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/models/qnn.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_g

Test Accuracy of the model: 443.75


In [21]:
print(test_results)

4.4375
